In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
sys.path.append('../experiments')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
from neuro.features import qa_questions, feature_spaces
from neuro.data import story_names
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge
import random
import json
import joblib
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
from tqdm import tqdm
from neuro.features.questions.gpt4 import QS_35_STABLE
from collections import defaultdict
from neuro.features.feature_spaces import get_gpt4_qa_embs_cached
fit_encoding = __import__('02_fit_encoding')

### Set up data for labeling

In [37]:
story_names_list = ['wheretheressmoke']
print('loaded', len(story_names_list), 'stories')
# wordseqs = feature_spaces.get_story_wordseqs(story_names_train)
wordseqs = load_story_wordseqs_huge(story_names_list)

ngrams_list_total = []
for story in story_names_list:
    ngrams_list = feature_spaces.get_ngrams_list_main(
        wordseqs[story], num_ngrams_context=10)
    # print(ngrams_list[:10])
    ngrams_list_total.extend(ngrams_list)

# questions = get_merged_questions_v3_boostexamples()
questions = QS_35_STABLE

loaded 1 stories


In [38]:
answers = np.full((len(ngrams_list_total), len(questions)), fill_value='')
df = pd.DataFrame(answers, columns=questions)
df['text'] = ngrams_list_total
# move question to first position
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.to_csv('ngrams.csv')

In [39]:
embs_gpt4 = get_gpt4_qa_embs_cached(
    story_name=story_names_list[0], questions=questions, qa_questions_version=None)
embs_gpt4 = pd.DataFrame(embs_gpt4, columns=questions, index=ngrams_list_total)
embs_gpt4.to_csv('qa_gpt4.csv')

In [40]:
embs_human1 = pd.read_csv('qa_human1.csv')
cols = [c for c in embs_human1.columns if not c in ['text', 'Unnamed: 0']]
assert cols == questions
assert embs_gpt4.shape[0] == embs_human1.shape[0]

In [ ]:
neuro.features.

In [ ]:
accs_dict = defaultdict(list)
for i, q in enumerate(questions):
    notna = embs_human1[q].notna()
    accs_dict['acc'].append(
        np.mean(embs_gpt4[q][notna] == embs_human1[q][notna]))
    accs_dict['acc_baseline'].append(np.mean(embs_human1[q][notna]))

In [ ]:
accs = pd.DataFrame(accs_dict)
accs['acc_baseline_majority'] = accs['acc_baseline'].apply(
    lambda x: max(x, 1-x))

# seaborn plot distributions
# sns.swarmplot(data=accs, color='black')
sns.barplot(data=accs)

In [ ]:
accs.style.background_gradient(cmap='coolwarm', axis=0)